In [1]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'blazingtext/supervised' #Replace with the prefix under which you want to store the data if needed

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::905418175678:role/service-role/AmazonSageMaker-ExecutionRole-20240501T190724
sagemaker-us-east-1-905418175678


## Data Preparation

Now we'll download a dataset from the web on which we want to train the text classification model. BlazingText expects a single preprocessed text file with space separated tokens and each line of the file should contain a single sentence and the corresponding label(s) prefixed by " __ label __ ".

In this example, let us train the text classification model on the [DBPedia Ontology Dataset](https://www.dbpedia.org/#2) as done by Zhang et al. The DBpedia ontology dataset is constructed by picking 14 nonoverlapping classes from DBpedia 2014. It has 560,000 training samples and 70,000 testing samples. The fields we used for this dataset contain title and abstract of each Wikipedia article.

In [2]:
!wget https://github.com/saurabh3949/Text-Classification-Datasets/raw/master/dbpedia_csv.tar.gz

--2024-05-01 18:22:40--  https://github.com/saurabh3949/Text-Classification-Datasets/raw/master/dbpedia_csv.tar.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/saurabh3949/Text-Classification-Datasets/master/dbpedia_csv.tar.gz [following]
--2024-05-01 18:22:40--  https://raw.githubusercontent.com/saurabh3949/Text-Classification-Datasets/master/dbpedia_csv.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68431223 (65M) [application/octet-stream]
Saving to: ‘dbpedia_csv.tar.gz’

100%[======================================>] 68,431,223   228MB/s   in 0.3s   

2024-05-01 18:22:41 (228 MB/s)

In [3]:
!tar -xzvf dbpedia_csv.tar.gz

dbpedia_csv/
dbpedia_csv/test.csv
dbpedia_csv/classes.txt
dbpedia_csv/train.csv
dbpedia_csv/readme.txt


#### Let us inspect the dataset and the classes to get some understanding about how the data and the label is provided in the dataset.

In [4]:
!head dbpedia_csv/train.csv -n 3

1,"E. D. Abbott Ltd"," Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972."
1,"Schwan-Stabilo"," Schwan-STABILO is a German maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. It is the world's largest manufacturer of highlighter pens Stabilo Boss."
1,"Q-workshop"," Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). They also run an online retail store and maintainan active forum community.Q-workshop was established in 2001 by Patryk Strzelewicz – a student from Poznań. Initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablishe

In [6]:
# check this out as a pandas df
import pandas as pd
train_df = pd.read_csv('dbpedia_csv/train.csv', header=None)
train_df.head(5)

,0,1,2
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...


#### Print the labels file (classes.txt) to see all possible labels followed by creating an index to label mapping.

In [7]:
!cat dbpedia_csv/classes.txt

Company
EducationalInstitution
Artist
Athlete
OfficeHolder
MeanOfTransportation
Building
NaturalPlace
Village
Animal
Plant
Album
Film
WrittenWork


#### Creating the mapping from integer indices to class label which will later be used to retrieve the actual class name during inference.

In [8]:
index_to_label = {} 
with open("dbpedia_csv/classes.txt") as f:
    for i,label in enumerate(f.readlines()):
        index_to_label[str(i+1)] = label.strip()
print(index_to_label)

{'1': 'Company', '2': 'EducationalInstitution', '3': 'Artist', '4': 'Athlete', '5': 'OfficeHolder', '6': 'MeanOfTransportation', '7': 'Building', '8': 'NaturalPlace', '9': 'Village', '10': 'Animal', '11': 'Plant', '12': 'Album', '13': 'Film', '14': 'WrittenWork'}


In [9]:
# note that the keys are strings, not integers.
index_to_label['1']

'Company'

### Data Preprocessing

We need to preprocess the training data into space separated tokenized text format which can be consumed by BlazingText algorithm. Also, as mentioned previously, the class label(s) should be prefixed with __ label __ and it should be present in the same line along with the original sentence. We'll use nltk library to tokenize the input sentences from DBPedia dataset.

Download the nltk tokenizer and other libraries

In [10]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
def transform_instance(row):
    cur_row = []
    label = "__label__" + index_to_label[row[0]]  #Prefix the index-ed label with __label__
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

In [12]:
# remember, there are 3 columns in the dataframe.
train_df.loc[:2]

,0,1,2
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...


In [13]:
# try applying our function to a few rows, just to see what it can do
row = train_df.iloc[0].values
row

array([1, 'E. D. Abbott Ltd',
       ' Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.'],
      dtype=object)

In [14]:
label = "__label__" + index_to_label[str(row[0])]
label

'__label__Company'

In [15]:
nltk.word_tokenize(row[1].lower())

['e.', 'd.', 'abbott', 'ltd']

In [16]:
nltk.word_tokenize(row[2].lower())

['abbott',
 'of',
 'farnham',
 'e',
 'd',
 'abbott',
 'limited',
 'was',
 'a',
 'british',
 'coachbuilding',
 'business',
 'based',
 'in',
 'farnham',
 'surrey',
 'trading',
 'under',
 'that',
 'name',
 'from',
 '1929.',
 'a',
 'major',
 'part',
 'of',
 'their',
 'output',
 'was',
 'under',
 'sub-contract',
 'to',
 'motor',
 'vehicle',
 'manufacturers',
 '.',
 'their',
 'business',
 'closed',
 'in',
 '1972',
 '.']

In [18]:
# put it all together
cur_row = []
label = "__label__" + index_to_label[str(row[0])]  #Prefix the index-ed label with __label__
cur_row.append(label)
cur_row.extend(nltk.word_tokenize(row[1].lower()))
cur_row.extend(nltk.word_tokenize(row[2].lower()))
cur_row

['__label__Company',
 'e.',
 'd.',
 'abbott',
 'ltd',
 'abbott',
 'of',
 'farnham',
 'e',
 'd',
 'abbott',
 'limited',
 'was',
 'a',
 'british',
 'coachbuilding',
 'business',
 'based',
 'in',
 'farnham',
 'surrey',
 'trading',
 'under',
 'that',
 'name',
 'from',
 '1929.',
 'a',
 'major',
 'part',
 'of',
 'their',
 'output',
 'was',
 'under',
 'sub-contract',
 'to',
 'motor',
 'vehicle',
 'manufacturers',
 '.',
 'their',
 'business',
 'closed',
 'in',
 '1972',
 '.']

In [20]:
# let's see if we can build our own tokenizer so we don't have to use NLTK.
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

In [21]:
print(row[2])
review_to_words(row[2])

 Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.


' abbott of farnham e d abbott limited was a british coachbuilding business based in farnham surrey trading under that name from 1929 a major part of their output was under sub contract to motor vehicle manufacturers their business closed in 1972'

#### The transform_instance will be applied to each data instance in parallel using python's multiprocessing module

In [22]:
def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, 'r') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[:int(keep*len(all_rows))]
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_instance, all_rows)
    pool.close() 
    pool.join()
    # blazing text requires space-separated word tokens.
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n') # notice the delimiter.
        csv_writer.writerows(transformed_rows)

In [23]:
%%time

# Preparing the training dataset

# Since preprocessing the whole dataset might take a couple of mintutes,
# we keep 20% of the training dataset for this demo.
# Set keep to 1 if you want to use the complete dataset
preprocess('dbpedia_csv/train.csv', 'dbpedia.train', keep=.2)
        
# Preparing the validation dataset        
preprocess('dbpedia_csv/test.csv', 'dbpedia.validation')

CPU times: user 8.57 s, sys: 1.08 s, total: 9.65 s
Wall time: 1min 22s


#### Need to upload it to S3 so that it can be consumed by SageMaker to execute training jobs. Using Python SDK to upload these two files to the bucket and prefix location that we have set above.

In [24]:
%%time

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

sess.upload_data(path='dbpedia.train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='dbpedia.validation', bucket=bucket, key_prefix=validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

CPU times: user 408 ms, sys: 181 ms, total: 589 ms
Wall time: 1.02 s


#### Now need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's traning job.

In [25]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
s3_output_location

's3://sagemaker-us-east-1-905418175678/blazingtext/supervised/output'

### Training

Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a `sageMaker.estimator.Estimator` object. This estimator will launch the training job.

In [26]:
region_name = boto3.Session().region_name
print(region_name)

us-east-1


In [27]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:1 (us-east-1)


### Training the BlazingText model for supervised text classification


Similar to the original implementation of [Word2Vec](https://arxiv.org/pdf/1301.3781), SageMaker BlazingText provides an efficient implementation of the continuous bag-of-words (CBOW) and skip-gram architectures using Negative Sampling, on CPUs and additionally on GPU[s]. The GPU implementation uses highly optimized CUDA kernels. [Scaling and Accelerating Word2Vec using Multiple GPUs](https://dl.acm.org/doi/10.1145/3146347.3146354).

BlazingText also supports a supervised mode for text classification. It extends the FastText text classifier to leverage GPU acceleration using custom CUDA kernels. The model can be trained on more than a billion words in a couple of minutes using a multi-core CPU or a GPU, while achieving performance on par with the state-of-the-art deep learning text classification algorithms. [Algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html).

Let's define the SageMaker `Estimator` with resource configurations and hyperparameters to train Text Classification on DBPedia dataset, using "supervised" mode on a `ml.t2.medium` instance.

In [36]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.m4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [37]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

#### The hyper-parameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the sagemaker.session.s3_input objects from our data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [38]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


The `Estimator` object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `Estimator` classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. Therefore it might be a few minutes before we start getting training logs for our training jobs. The data logs will also print out Accuracy on the validation data for every epoch after training job has executed `min_epochs`. This metric is a proxy for the quality of the algorithm.

In [39]:
bt_model.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating training-job with name: blazingtext-2024-05-01-19-16-47-113


2024-05-01 19:16:47 Starting - Starting the training job...
2024-05-01 19:17:02 Starting - Preparing the instances for training...
2024-05-01 19:17:45 Downloading - Downloading input data...
2024-05-01 19:18:10 Training - Training image download completed. Training in progress...Arguments: train
[05/01/2024 19:18:26 WARNING 140176013698880] Loggers have already been setup.
[05/01/2024 19:18:26 WARNING 140176013698880] Loggers have already been setup.
/opt/amazon/python3.8/lib/python3.8/subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
[05/01/2024 19:18:26 INFO 140176013698880] nvidia-smi took: 0.025201797485351562 secs to identify 0 gpus
[05/01/2024 19:18:26 INFO 140176013698880] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[05/01/2024 19:18:26 INFO 140176013698880] Processing /opt/ml/i

#### Hosting / Inference using Serverless Endpoint

https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-serverless-inference

In [67]:
from sagemaker.serverless import ServerlessInferenceConfig

# Create an empty ServerlessInferenceConfig object to use default values
serverless_config = ServerlessInferenceConfig()

#### The ServerlessInferenceConfig in the estimator’s deploy() method to deploy a serverless endpoint:

In [68]:
# Deploys the model that was generated by fit() to a SageMaker serverless endpoint
text_classifier = bt_model.deploy(serverless_inference_config=serverless_config)


# text_classifier = bt_model.deploy(initial_instance_count = 1,
#                                   instance_type = 'ml.t2.medium')


INFO:sagemaker:Creating model with name: blazingtext-2024-05-01-20-23-38-097
INFO:sagemaker:Creating endpoint-config with name blazingtext-2024-05-01-20-23-38-097
INFO:sagemaker:Creating endpoint with name blazingtext-2024-05-01-20-23-38-097


---!

In [69]:
print(text_classifier.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


blazingtext-2024-05-01-20-23-38-097


#### Use JSON format for inference

BlazingText supports `application/json` as the content-type for inference. The payload should contain a list of sentences with the key as "instances" while being passed to the endpoint.

In [42]:
sentences = ["Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft.",
            "Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick ."]

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [' '.join(nltk.word_tokenize(sent)) for sent in sentences]
tokenized_sentences

['Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft .',
 'Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick .']

In [43]:
payload = {"instances" : tokenized_sentences}
payload

{'instances': ['Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft .',
  'Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick .']}

In [44]:
json.dumps(payload)

'{"instances": ["Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft .", "Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick ."]}'

In [45]:
response = text_classifier.predict(json.dumps(payload), initial_args={'ContentType': 'application/json'})
response

b'[{"label": ["__label__Company"], "prob": [0.9987420439720154]}, {"label": ["__label__EducationalInstitution"], "prob": [0.9937962293624878]}]'

In [46]:
predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

[
  {
    "label": [
      "__label__Company"
    ],
    "prob": [
      0.9987420439720154
    ]
  },
  {
    "label": [
      "__label__EducationalInstitution"
    ],
    "prob": [
      0.9937962293624878
    ]
  }
]


By default, the model will return only one prediction, the one with the highest probability. For retrieving the top k predictions, you can set k in the configuration as shown below:

In [47]:
payload = {"instances" : tokenized_sentences,
          "configuration": {"k": 2}}

response = text_classifier.predict(json.dumps(payload), initial_args={'ContentType': 'application/json'})

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

[
  {
    "label": [
      "__label__Company",
      "__label__MeanOfTransportation"
    ],
    "prob": [
      0.9987420439720154,
      0.001049356535077095
    ]
  },
  {
    "label": [
      "__label__EducationalInstitution",
      "__label__Village"
    ],
    "prob": [
      0.9937962293624878,
      0.00393278943374753
    ]
  }
]


#### Lambda Function

In [65]:
import boto3
runtime = boto3.Session().client('sagemaker-runtime')
print(runtime)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [66]:
print(text_classifier.endpoint)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


blazingtext-2024-05-01-19-24-59-504


#### The SageMaker runtime and the name of our endpoint, we can invoke the endpoint and send it the test data.

In [58]:
sentences = ["The Empire State Building is a 102-story[c] Art Deco skyscraper in Midtown Manhattan in New York City. It was designed by Shreve, Lamb & Harmon and built from 1930 to 1931. Its name is derived from Empire State, the nickname of the state of New York. The building has a roof height of 1,250 feet (380 m) and stands a total of 1,454 feet (443.2 m) tall, including its antenna. The Empire State Building stood as the world's tallest building until the construction of the World Trade Center in 1970; following its collapse in the September 11, 2001 attacks, the Empire State Building was again the city's tallest skyscraper until 2012. As of 2020, the building is the seventh-tallest building in New York City, the ninth-tallest completed skyscraper in the United States, the 48th-tallest in the world, and the fifth-tallest freestanding structure in the Americas."]

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [' '.join(nltk.word_tokenize(sent)) for sent in sentences]
tokenized_sentences

["The Empire State Building is a 102-story [ c ] Art Deco skyscraper in Midtown Manhattan in New York City . It was designed by Shreve , Lamb & Harmon and built from 1930 to 1931 . Its name is derived from Empire State , the nickname of the state of New York . The building has a roof height of 1,250 feet ( 380 m ) and stands a total of 1,454 feet ( 443.2 m ) tall , including its antenna . The Empire State Building stood as the world 's tallest building until the construction of the World Trade Center in 1970 ; following its collapse in the September 11 , 2001 attacks , the Empire State Building was again the city 's tallest skyscraper until 2012 . As of 2020 , the building is the seventh-tallest building in New York City , the ninth-tallest completed skyscraper in the United States , the 48th-tallest in the world , and the fifth-tallest freestanding structure in the Americas ."]

In [59]:
payload = {"instances" : tokenized_sentences}
print(json.dumps(payload))

# response = text_classifier.predict(json.dumps(payload))

{"instances": ["The Empire State Building is a 102-story [ c ] Art Deco skyscraper in Midtown Manhattan in New York City . It was designed by Shreve , Lamb & Harmon and built from 1930 to 1931 . Its name is derived from Empire State , the nickname of the state of New York . The building has a roof height of 1,250 feet ( 380 m ) and stands a total of 1,454 feet ( 443.2 m ) tall , including its antenna . The Empire State Building stood as the world 's tallest building until the construction of the World Trade Center in 1970 ; following its collapse in the September 11 , 2001 attacks , the Empire State Building was again the city 's tallest skyscraper until 2012 . As of 2020 , the building is the seventh-tallest building in New York City , the ninth-tallest completed skyscraper in the United States , the 48th-tallest in the world , and the fifth-tallest freestanding structure in the Americas ."]}


In [60]:
response = runtime.invoke_endpoint(EndpointName = text_classifier.endpoint, # The name of the endpoint we created
                                       ContentType = 'application/json', # The data format that is expected
                                       Body = json.dumps(payload))

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [61]:
response

{'ResponseMetadata': {'RequestId': 'c3e88ce6-1394-4e39-93aa-9f3a2fe6725a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c3e88ce6-1394-4e39-93aa-9f3a2fe6725a',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Wed, 01 May 2024 20:16:49 GMT',
   'content-type': 'application/json',
   'content-length': '64',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f55c7a7ad10>}

In [62]:
output = json.loads(response['Body'].read().decode('utf-8'))
output[0]['prob'][0]

0.9408228397369385

In [63]:
output[0]['label'][0].split('__label__')[1]

'Building'

#### Stop / Close the Endpoint (Optional)

Finally, we should delete the endpoint before we close the notebook if we don't need to keep the endpoint running for serving realtime predictions.

In [64]:
# sess.delete_endpoint(text_classifier.endpoint)